In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.special import multigammaln
from scipy.stats import invwishart, invgamma

In [ ]:
cov_matrix = np.array([[1, 0], [0, 2]])

### 1) Estimating Covariance using the Maximum Likelihood approach
 

In [ ]:
for n in [10,100,1000]:
  np.random.seed(1234)
  y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
  cov_estimate = np.matmul(np.transpose(y),y)/n
  print("\nEstimated covariance matrix for n =",n)
  print(cov_estimate)
  print(f"MSE for n = {n} is ",np.mean(np.square(cov_estimate-cov_matrix)))


Estimated covariance matrix for n = 10
[[ 1.52839538 -0.69924025]
 [-0.69924025  1.55723106]]
MSE for n = 10 is  0.3632799625398799

Estimated covariance matrix for n = 100
[[ 1.15268269 -0.25000989]
 [-0.25000989  1.60006159]]
MSE for n = 100 is  0.07706815796164049

Estimated covariance matrix for n = 1000
[[ 0.99585968 -0.03801164]
 [-0.03801164  1.870427  ]]
MSE for n = 1000 is  0.004924018986004798


### 2) Bayesian estimation with  Inverse Wishart distribution as prior

In [ ]:
v_0 = 5
d = 2
delta_0 = np.array([[4, 0], [0, 5]])

for n in [10,100,1000]:
  np.random.seed(1234)
  y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
  delta_n = delta_0+np.matmul(np.transpose(y),y)
  v_n = v_0+n
  cov_estimate = delta_n/(v_n+d+1)
  print("\nEstimated covariance matrix for n =",n) 
  print(cov_estimate)
  print(f"MSE for n = {n} is ",np.mean(np.square(cov_estimate-cov_matrix)))


Estimated covariance matrix for n = 10
[[ 1.07133076 -0.3884668 ]
 [-0.3884668   1.14290614]]
MSE for n = 10 is  0.26037771752735406

Estimated covariance matrix for n = 100
[[ 1.10433583 -0.23149064]
 [-0.23149064  1.5278348 ]]
MSE for n = 100 is  0.08525044303918006

Estimated covariance matrix for n = 1000
[[ 0.99192429 -0.03770997]
 [-0.03770997  1.86054265]]
MSE for n = 1000 is  0.005589412821708227


### 3) Use of non informative prior

In [ ]:
for n in [10,100,1000]:
  np.random.seed(1234)
  y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
  S = np.matmul(np.transpose(y),y)
  cov_estimate = S/(n+4)
  print(f"\nEstimated covariance matrix for n = {n} for non-informative Jeffrey’s prior") 
  print(cov_estimate)
  print(f"MSE for n = {n} is ",np.mean(np.square(cov_estimate-cov_matrix))) 


Estimated covariance matrix for n = 10 for non-informative Jeffrey’s prior
[[ 1.09171098 -0.49945732]
 [-0.49945732  1.1123079 ]]
MSE for n = 10 is  0.3238308488109908

Estimated covariance matrix for n = 100 for non-informative Jeffrey’s prior
[[ 1.10834874 -0.24039413]
 [-0.24039413  1.53852076]]
MSE for n = 100 is  0.08507030378630773

Estimated covariance matrix for n = 1000 for non-informative Jeffrey’s prior
[[ 0.99189211 -0.0378602 ]
 [-0.0378602   1.8629751 ]]
MSE for n = 1000 is  0.005427088104887309


In [ ]:
for n in [10,100,1000]:
  np.random.seed(1234)
  y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
  S = np.matmul(np.transpose(y),y)
  cov_estimate = S/(n+3)
  print(f"\nEstimated covariance matrix for n = {n} for independence-Jeffrey’s prior") 
  print(cov_estimate)
  print(f"MSE for n = {n} is ",np.mean(np.square(cov_estimate-cov_matrix)))  


Estimated covariance matrix for n = 10 for independence-Jeffrey’s prior
[[ 1.17568875 -0.53787711]
 [-0.53787711  1.19787005]]
MSE for n = 10 is  0.31322564401605113

Estimated covariance matrix for n = 100 for independence-Jeffrey’s prior
[[ 1.11910941 -0.24272805]
 [-0.24272805  1.55345785]]
MSE for n = 100 is  0.08285518891682812

Estimated covariance matrix for n = 1000 for independence-Jeffrey’s prior
[[ 0.99288104 -0.03789795]
 [-0.03789795  1.8648325 ]]
MSE for n = 1000 is  0.005298360629828628


### 4) Monte carlo bayesian estimation

In [ ]:
delta_0 = np.array([[4,0],[0,5]])
v_0 = 5

def exp_term(y, sigma):    
    return -0.5 * (y@np.linalg.inv(sigma)*y).sum()

for n in [10,100,1000]:
    np.random.seed(1234)
    y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
    for m in [1000,10000,100000]:
        sigmas = invwishart.rvs(df=v_0,  scale=delta_0, size=m, random_state=1234)
        dets_list = list(map(np.linalg.det,sigmas))
        dets_array = np.array(dets_list)
        abs_dets = np.abs(dets_array)

        exps = np.array([exp_term(y, sigma) for sigma in sigmas])
        exp_pow_and_det = exps - 0.5 * n * np.log(abs_dets)
        exp_pow_and_det = exp_pow_and_det - np.max(exp_pow_and_det)
        denominator_terms = np.exp(exp_pow_and_det)  
        numerator_terms = np.multiply(sigmas, denominator_terms[:, None, None])  
        numerator = np.mean(numerator_terms, axis=0)
        denominator = np.mean(denominator_terms, axis=0)
        mc_estimate = numerator/denominator
        print(f'\nMonte carlo estimate or A for n={n} and m={m} is :\n{mc_estimate}')
        print(f"MSE for this configuration is ",np.mean(np.square(mc_estimate-cov_matrix))) 



Monte carlo estimate or A for n=10 and m=1000 is :
[[ 1.6186982  -0.53358614]
 [-0.53358614  1.64849999]]
MSE for this configuration is  0.2689420105654011

Monte carlo estimate or A for n=10 and m=10000 is :
[[ 1.59704462 -0.5718405 ]
 [-0.5718405   1.71323759]]
MSE for this configuration is  0.2731745182578328

Monte carlo estimate or A for n=10 and m=100000 is :
[[ 1.61210533 -0.58417297]
 [-0.58417297  1.71036704]]
MSE for this configuration is  0.285269072796079

Monte carlo estimate or A for n=100 and m=1000 is :
[[ 1.17361888 -0.2370108 ]
 [-0.2370108   1.59272821]]
MSE for this configuration is  0.07709051401727435

Monte carlo estimate or A for n=100 and m=10000 is :
[[ 1.17121685 -0.25234526]
 [-0.25234526  1.62188775]]
MSE for this configuration is  0.07491008809268067

Monte carlo estimate or A for n=100 and m=100000 is :
[[ 1.1655761  -0.24044226]
 [-0.24044226  1.61440333]]
MSE for this configuration is  0.07293129904473297

Monte carlo estimate or A for n=1000 and m=100

In [ ]:
delta_0 = np.array([[2,0],[0,4]])
v_0 = 5

def exp_term(y, sigma):    
    return -0.5 * (y@np.linalg.inv(sigma)*y).sum()

for n in [10,100,1000]:
    np.random.seed(1234)
    y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
    for m in [1000,10000,100000]:
        sigmas = invwishart.rvs(df=v_0,  scale=delta_0, size=m,random_state=1234)
        dets_list = list(map(np.linalg.det,sigmas))
        dets_array = np.array(dets_list)
        abs_dets = np.abs(dets_array)

        exps = np.array([exp_term(y, sigma) for sigma in sigmas])
        exp_pow_and_det = exps - 0.5 * n * np.log(abs_dets)
        exp_pow_and_det = exp_pow_and_det - np.max(exp_pow_and_det)
        denominator_terms = np.exp(exp_pow_and_det)  
        numerator_terms = np.multiply(sigmas, denominator_terms[:, None, None])  
        numerator = np.mean(numerator_terms, axis=0)
        denominator = np.mean(denominator_terms, axis=0)
        mc_estimate = numerator/denominator
        print(f'\nMonte carlo estimate or A for n={n} and m={m} is :\n{mc_estimate}')
        print(f"MSE for this configuration is ",np.mean(np.square(mc_estimate-cov_matrix))) 



Monte carlo estimate or A for n=10 and m=1000 is :
[[ 1.45413455 -0.56460302]
 [-0.56460302  1.57251004]]
MSE for this configuration is  0.2566347495690538

Monte carlo estimate or A for n=10 and m=10000 is :
[[ 1.40917452 -0.55448462]
 [-0.55448462  1.5943145 ]]
MSE for this configuration is  0.23672772315786264

Monte carlo estimate or A for n=10 and m=100000 is :
[[ 1.44529152 -0.58773328]
 [-0.58773328  1.62384184]]
MSE for this configuration is  0.257660081186997

Monte carlo estimate or A for n=100 and m=1000 is :
[[ 1.13733212 -0.26076304]
 [-0.26076304  1.68117801]]
MSE for this configuration is  0.06412557574486913

Monte carlo estimate or A for n=100 and m=10000 is :
[[ 1.16431362 -0.26363005]
 [-0.26363005  1.58698195]]
MSE for this configuration is  0.08414612162736437

Monte carlo estimate or A for n=100 and m=100000 is :
[[ 1.14221605 -0.2436836 ]
 [-0.2436836   1.61050506]]
MSE for this configuration is  0.07267377558053453

Monte carlo estimate or A for n=1000 and m=10

### 5) Hierarchical Bayes estimation and Gibbs sampling

In [ ]:
d = 2
nu = 5
A1 = 0.05
A2 = 0.05
no_iterations = 1000

for n in [10,100,1000]:
    np.random.seed(1234)
    y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
    sum_of_product_terms = np.matmul(np.transpose(y),y)
    a1 = invgamma.rvs(1/2, scale=1/(A1**2))
    a2 = invgamma.rvs(1/2, scale=1/(A2**2))
    j=0
    while(j < no_iterations):
        sigma_scale = 2*nu*np.diag([1/a1,1/a2]) + sum_of_product_terms
        sigma = invwishart.rvs(df=nu+d+n-1,scale=sigma_scale, size=1)
        sigma_inverse = np.linalg.inv(sigma)
        a1 = invgamma.rvs((nu + n)/2, scale=nu *sigma_inverse[0, 0] + 1/(A1**2))
        a2 = invgamma.rvs((nu + n)/2, scale=nu *sigma_inverse[1, 1] + 1/(A2**2))
        j=j+1
    final_sigma = 2*nu*np.diag([1/a1, 1/a2]) + sum_of_product_terms
    nu_n = nu+d+n-1
    cov_estimate = final_sigma/(nu_n+d+1)  
    print(f'\nThe covariance estimate from Gibbs sampling for n={n} is :\n{cov_estimate}')
    print(f"MSE for n = {n} is ",np.mean(np.square(cov_estimate-cov_matrix)))


The covariance estimate from Gibbs sampling for n=10 is :
[[ 0.82155445 -0.36802118]
 [-0.36802118  0.82609591]]
MSE for n = 10 is  0.42019320058345166

The covariance estimate from Gibbs sampling for n=100 is :
[[ 1.06995358 -0.22936687]
 [-0.22936687  1.48311169]]
MSE for n = 100 is  0.09432133958432733

The covariance estimate from Gibbs sampling for n=1000 is :
[[ 1.00048121 -0.03767259]
 [-0.03767259  1.86549152]]
MSE for n = 1000 is  0.005232802605366902


### 6) Empirical Bayes

In [ ]:
d = 2
threshold = 1e-6 

for n in [10,100,1000]:
    np.random.seed(1234)
    y = np.random.multivariate_normal(np.zeros(2), cov_matrix, n)
    sum_of_product_terms = np.matmul(np.transpose(y),y)
    error = np.ones(1)
    nu = 1 
    while np.abs(error) > threshold:
        temp = np.array([1/(nu+n-i-j-1 + 1e-7) for i in range(1, n+1) for j in range(1, n+1)])
        derivative = np.log((nu+n)/nu) - np.sum(temp)
        double_derivative = 1/(nu + n) - 1/nu + np.sum(np.square(temp))
        error = derivative/double_derivative
        nu = nu - error
    nu_opt = nu
    delta_opt = (nu_opt*sum_of_product_terms)/n
    nu_n = nu_opt+n
    delta_n = delta_opt + sum_of_product_terms
    cov_estimate = delta_n/(nu_n+d+1)
    print(f'\nThe estimated covariance from Emperical Bayes method for n={n} is:\n{cov_estimate}')
    print(f"MSE for n = {n} is ",np.mean(np.square(cov_estimate-cov_matrix)))


The estimated covariance from Emperical Bayes method for n=10 is:
[[ 1.20088208 -0.54940305]
 [-0.54940305  1.22353869]]
MSE for n = 10 is  0.31173329995130933

The estimated covariance from Emperical Bayes method for n=100 is:
[[ 1.11943223 -0.24279807]
 [-0.24279807  1.55390596]]
MSE for n = 100 is  0.082791437693855

The estimated covariance from Emperical Bayes method for n=1000 is:
[[ 0.992884   -0.03789806]
 [-0.03789806  1.86483807]]
MSE for n = 1000 is  0.005297977780410795
